# Unusual Rotas

All of our rota examples so far have had very reasonable date representations. Let's look at an another example [unusual1.xlsx](unusual1.xlsx):

| A | B | C |
|:---:|:---:|:---:|
| OCT |  |  |
| 1 | SpR3 |  |
| 2 | SpR1 |  |
| 3 | SpR6(JR) | SpR5 doing 5th Oct |
| 4 | SpR2 |  |
| 5 | SpR5 | SpR6 doing 3rd Oct |
| 6 | SpR3 |  |
| 7 | SpR2 | SpR4 doing 4th Nov  |
| 8 | SpR2 | SpR4 doing 5th Nov  |
| 9 | SpR5 |  |
| 10 | SpR2 |  |
| 11 | SpR4 | SpR6 doing 3 Nov |
| 12 | SpR3 |  |
| 13 | SpR2 | SpR4 doing 11 Oct |
| 14 | SpR1 |  |
| 15 | SpR1 |  |
| 16 | SpR2 |  |
| 17 | SpR6 |  |
| 18 | SpR3 |  |
| 19 | SpR4 |  |
| 20 | SpR6 | SpR1 doing 3rd November |
| 21 | SpR5 |  |
| 22 | SpR5 |  |
| 23 | SpR3 |  SpR6 Doing 24 oct |
| 24 | SpR6 | SpR3 doing 23rd Oct |
| 25 | SpR4 |  |
| 26 | SpR1 |  |
| 27 | SpR5 |  |
| 28 | SpR2 |  |
| 29 | SpR2 |  |
| 30 | SpR5 | UCH 3 doing |
| 31 | SpR4 |  |
| NOV |  |  |
| 1 | SpR1 |  |

## First steps - loading the rota
OK, so how do we deal with a rota like this...? Let's first try loading it and see what we've got...

In [2]:
from xlrd_helper import DictReader
rows = []

with open('unusual1.xlsx', 'rb') as f:
    rows = [ row for row in DictReader(f)]
    print(rows[0].keys())
    print(rows[0])

odict_keys(['Jun', ''])
OrderedDict([('Jun', '1'), ('', '')])


Now looking carefully we see that the fieldnames aren't quite right, (neither is the data quite what we were expecting either - but we'll get to that). It turns out that this rota doesn't have headers. 

We have two options: We can drop down to using the Reader implementation or we could pass in some fieldnames. Let's look at passing in some fieldnames. I'll also pass in a `restkey` which will allow us to pick up anything placed in other columns.

In [3]:
from xlrd_helper import DictReader
rows = []
fieldnames = ['date', 'oncall', 'additional']

with open('unusual1.xlsx', 'rb') as f:
    rows = [ row for row in DictReader(f, fieldnames=fieldnames, 
                                      restkey='other')]
    print(rows[0].keys())
    print(rows[0])

odict_keys(['date', 'oncall', 'additional', 'other'])
OrderedDict([('date', 'Jun'), ('oncall', ''), ('additional', ''), ('other', ['', '', '', ''])])


Now, let's go back to why the first piece of data is 'Jun' when we were expecting 'Oct'...

It turns out that if you look at the excel file, the first 503 rows are hidden. Getting the information about hidden rows requires adding a few options to the `DictReader` and querying the workbook ourselves,

In [4]:
from xlrd_helper import DictReader
rows = []
fieldnames = ['date', 'oncall', 'additional']

try:
    with open('unusual1.xlsx', 'rb') as f:
        dr = DictReader(f, fieldnames=fieldnames, restkey='other', formatting_info=True)
        rows = [ row for row in dr]
        print(rows[0].keys())
        print(rows[0])
        print(dr.reader.sheet.rowinfo_map)
except NotImplementedError:
    print('Formatting info not implemented in xlrd for XLSX')

Formatting info not implemented in xlrd for XLSX


Now, it turns out there is another package out there for doing this: `openpyxl`. However, we should probably consider whether we want to go down this path... If the rota co-ordinator is just hiding rows then it means that the starting point for the file is unlikely to change - whereas the first hidden row is likely to change. 

So thinking on we probably don't need to know whether a row is hidden or not. (It may be helpful in future though.)


## Looking at the first column
Let's look at the contents of the first column.

In [5]:
from xlrd_helper import DictReader

fieldnames = ['date', 'oncall', 'additional']

with open('unusual1.xlsx', 'rb') as f:
    dr = DictReader(f, fieldnames=fieldnames, restkey='other')
    rows = [ row for row in dr ]
    dates = [ row['date'] for row in rows]

print(dates)

['Jun', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 'Jul', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', 'Aug', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', 'Sept', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 'Oct', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', 'Nov', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '

There are 4 things in there: a month indicator, a day, a year indicator, or a date

In [6]:
import re 
day_match = re.compile('^\d\d?$')
year_match = re.compile('^\d\d\d\d$')
date_match = re.compile('^\d\d\d\d/\d\d/\d\d$')
days, years, actual_dates, others = [], [], [], []
for i, d in enumerate(dates):
    if day_match.match(d):
        days.append((i, d))
    elif year_match.match(d):
        years.append((i, d))
    elif date_match.match(d):
        actual_dates.append((i, d))
    else:
        others.append((i, d))

print ('No. of days:\t%3d' % len(days))
print ('No. of years:\t%3d' % len(years))
print ('No. of dates:\t%3d' % len(actual_dates))
print ('Others:\t\t%3d' % len(others))

No. of days:	579
No. of years:	  2
No. of dates:	420
Others:		 19


So we should just check those others and make sure that they actually are months and whilst we are at it let's check that there is only on-call information in the rows that have days or dates.

In [7]:
for i, o in others:
    print (i, o)
    
print(all((all((rows[i]['oncall'] == '' for i,_ in others)),
           all((rows[i]['oncall'] == '' for i,_ in years)),
           all((rows[i]['oncall'] != '' for i,_ in days)),
           all((rows[i]['oncall'] != '' for i,_ in actual_dates)))))

0 Jun
31 Jul
63 Aug
95 Sept
126 Oct
158 Nov
189 Dec
221 jAN
253 FEB
282 MAR
314 APR
345 MAY
377 JUN
408 JUL
440 AUG
472 SEP
503 OCT
535 NOV
566 DEC
True


## Parsing the first column
How can we parse this? Either, the row contains an actual date, or it's one of the following: A year, a month or a day. So if we have a good idea of the preceding date we can just adjust our date and in fact `dateutil.parser.parse` will interpret a given date string in the context of another default date.

Looking at the column data carefully we can see that the first row refers to June 2016. We could work backwards from the dates but let's go forwards for the moment, so let's set the our default date as June 1st 2016.

In [8]:
from datetime import date

from dateutil.parser import parse

oncall = {}

today = date(2016, 6, 1)
for i, row in enumerate(rows):
    d = row['date']
    c = row['oncall']
    a = row['additional']

    # Parse our new date in the context of the previous date
    today = parse(d, default=today)
    
    # If we're setting an oncall person
    if c != '':
        if today in oncall:
            print('Duplicate: ', today, i, d)
        else:
            oncall[today] = (c, a)

ValueError: day is out of range for month

Now that's strange... How did that happened? Let's catch that error and add some logging:

In [9]:
from datetime import date

from dateutil.parser import parse

oncall = {}

today = date(2016, 6, 1)
for i, row in enumerate(rows):
    d = row['date']
    c = row['oncall']
    a = row['additional']

    try:
        # Parse our new date in the context of the previous date
        yesterday = today
        today = parse(d, default=yesterday)

        # If we're setting an oncall person
        if c != '':
            if today in oncall:
                print('Duplicate: ', today, i, d)
            else:
                oncall[today] = (c, a)
    except ValueError as e:
        print(e)
        print('Row %d with day value: %s, (Value Above: %s, Below: %s)' % (i, d, rows[i -1]['date'], rows[i + 1]['date']))
        

day is out of range for month
Row 10 with day value: 0, (Value Above: 9, Below: 11)
day is out of range for month
Row 41 with day value: 0, (Value Above: 9, Below: 11)
day is out of range for month
Row 73 with day value: 0, (Value Above: 9, Below: 11)
day is out of range for month
Row 105 with day value: 0, (Value Above: 9, Below: 11)
day is out of range for month
Row 136 with day value: 0, (Value Above: 9, Below: 11)
Duplicate:  2016-06-01 378 1
Duplicate:  2016-06-02 379 2
Duplicate:  2016-06-03 380 3
Duplicate:  2016-06-04 381 4
Duplicate:  2016-06-05 382 5
Duplicate:  2016-06-06 383 6
Duplicate:  2016-06-07 384 7
Duplicate:  2016-06-08 385 8
Duplicate:  2016-06-09 386 9
Duplicate:  2016-06-11 388 11
Duplicate:  2016-06-12 389 12
Duplicate:  2016-06-13 390 13
Duplicate:  2016-06-14 391 14
Duplicate:  2016-06-15 392 15
Duplicate:  2016-06-16 393 16
Duplicate:  2016-06-17 394 17
Duplicate:  2016-06-18 395 18
Duplicate:  2016-06-19 396 19
Duplicate:  2016-06-20 397 20
Duplicate:  2016-

The observant will have noticed that for some reason there are a couple of rows which have `0` instead of `10`. There's no other `0`s in the column.

Now understanding the duplicates requires a bit more thought. The first duplicate is in row 378 and the last duplicate is interpreted as 2016-12-31 in row 597. Now row 598 is `2018` - so I think we're not getting the change of year right. Let's look at what's supposed to happen when go over the year.

In [10]:
today = date(2016, 12, 31)
print(parse('Jan', default=today))

2016-01-31


Which obviously doesn't work. We'll just have to catch this case and manage the changeover ourselves.

In [11]:
from dateutil.parser import parse

oncall = {}

today = date(2016, 6, 1)
for i, row in enumerate(rows):
    d = row['date']
    c = row['oncall']
    a = row['additional']

    try:
        # Parse our new date in the context of the previous date
        yesterday = today
        if d == '0':
            d = '10'
        if today.month == 12 and today.day == 31:
            today = date(today.year + 1, today.month, today.day)
        today = parse(d, default=today)

        # If we're setting an oncall person
        if c != '':
            if today in oncall:
                print('Duplicate: ', today, i, d)
            else:
                oncall[today] = (c, a)
    except ValueError as e:
        print(e)
        print('Row %d with day value: %s, (Value Above: %s, Below: %s)' % (i, d, rows[i -1]['date'], rows[i + 1]['date']))


## Other considerations
The whole rota runs from 2016-2018. Obviously, it's not much help for us to have a rota for the whole of that range, so we would need to set some starting and end dates for the range. We may aswell have these hardcoded for the moment, but it's not hard to think of a way to pass these in as a parameter if you want.

## Creating the rota
So we could create a new reader from scratch, or we could instead adjust one of our previous readers. It's simpler to just adjust an old reader so let's do that.

Let's recall the strcture of the multi_rota3.py:

```python
### IMPORTS
...
### CONSTANTS
...
HOURS = { ... }
...
### FUNCTIONS
...
SPELLING_CORRECTIONS = { ... }
UNNECESSARY_ADDITIONAL_INFORMATION_RES = [ ... ]

def strip_unnecessary_information(name):
    ...

def autocorrect(name):
    ...

AM_PM_SPLIT_RE = re.compile('(.*) \(?(am)\)? (.*) \(?(pm)\)?')
def role_split(role_string):
    ...

def munge_role(name, role, row):
    ...

## Conversion functions 
def convert_to_date(date_str):
    ...

## Calendar functions
def create_calendar_for(name, job, role_rows_list):
    ...

def create_event_for(name, role, row):
    ...

## File reading functions
def read_csv(fname, handler, sheet, *args, **kwds):
    ...
                            
def read_excel(fname, handler, sheet=0, *args, **kwds):
    ...

def read(fname, handler, sheet=0, *args, **kwds):
    ...
                            
## Reading functions
def handle_rows(rows):
    ...

## Check last names functions
def check_last_names(nj_to_r_rows, directory):
    ...
                            
## Writing functions
def create_calendars(nj_to_r_rows, directory):
    ...

## Main function
def parse_file_and_create_calendars(fname, sheet, directory):
    from os.path import exists
    rows_data = read(fname, handle_rows, sheet)
    
    if not exists(directory):
        from os import makedirs
        makedirs(directory)
    check_last_names(rows_data, directory)
    create_calendars(rows_data, directory)

### MAIN
if __name__ == '__main__':
   ...
```

### `HOURS`
This is simple, our shifts are all day in this case

```python
HOURS = {
    'On-Call': {
        'duration': timedelta(days=1)
    },
}
```

### `BETWEEN` and `START_DAY`
We should add a default `BETWEEN` and `START_DAY`. 

```python
BETWEEN = (date(2017, 12, 6), date(2018, 3, 7))

START_DAY = date(2016,1,1)
```

### `SPELLING_CORRECTIONS` and `munge_role` etc.
We don't need to keep any of the role munging functions as there's no roles in this rota. We can probably keep the spelling correction functions as these may come in handy.

### `convert_to_date`
If we think about the way we parsed the rota above we already did the conversion - so we don't need this function either. (Of course we'll have to adjust any code that uses it.)

### `create_calendar_for`, `create_event_for` and `handle_rows`
In multi_rota3.py `handle_rows` returns a dictionary of name job pairs to a list of role and rows pairs. Previous iterations had a dictionary of name to rows.

Our above code creates a dictionary of dates to person on-call - so in order to keep it a simple change we should create a dictionary of name to list of days on-call with additional information.

In [15]:
def handle_rows(rows):
    """Store the rota information by name and job"""
    today = START_DAY
    on_call = {}

    for i, row in enumerate(rows):
        if row[0] == '0':
            row[0] = '10'
        try:
            if today.month == 12 and today.day == 31:
                today = date(today.year + 1, today.month, today.day)
            today = dateutil.parser.parse(row[0], default=today)
            if row[1] != '':
                if today in on_call:
                    print('Duplicate: ', today, row)
                else:
                    on_call[today] = (autocorrect(row[1]), row[2])
        except Exception:
            print('Weird row[', i, ']:', row)

    name_to_dates = defaultdict(list)

    for day in on_call:
        name, additional = on_call[day]
        name_to_dates[name].append((day, name, additional))
        name_to_dates['All'].append((day, name, additional))

    return name_to_dates

Which means we can change our `create_calendar_for` function to:

In [20]:
def create_calendar_for(name, dates, between):
    """Create a calendar for name in job using the provided rows"""
    # Create a basic iCalendar object
    cal = Calendar()

    # These two lines are required but you can change the prodid slightly
    cal.add('prodid', '-//hacksw/handcal/NONSGML v1.0//EN')
    cal.add('version', '2.0')

    # This means that your calendar gets a nice default name
    cal.add('x-wr-calname', 'Unusual-1 on-call rota for %s' % (name))

    # Now open the rota
    if name == 'All':
        for day, name, additional in dates:
            if (day >= between[0] and day < between[1]):
                if day.weekday() == 5:  # SAT
                    # Get a day off before
                    cal.add_component(
                        create_event_for('Lieu',
                                         day - timedelta(days=1),
                                         '',
                                         name))
                cal.add_component(create_event_for('On-Call',
                                                   day,
                                                   additional,
                                                   name))
                if day.weekday() < 4 or day.weekday() == 6:  # MON-THURS or SUN
                    # Get a day off afterwards
                    cal.add_component(
                        create_event_for('Lieu',
                                         day + timedelta(days=1),
                                         '',
                                         name))
    else:
        for day, name, additional in dates:
            # OK first of all create the on-call event for this day
            if (day >= between[0] and day < between[1]):
                if day.weekday() == 5:  # SAT
                    # Get a day off before
                    cal.add_component(
                        create_event_for('Lieu', day - timedelta(days=1)))
                cal.add_component(create_event_for('On-Call', day, additional))
                if day.weekday() < 4 or day.weekday() == 6:  # MON-THURS or SUN
                    # Get a day off afterwards
                    cal.add_component(
                        create_event_for('Lieu', day + timedelta(days=1)))
    return cal

Finally we have to adjust the `create_event_for` method.

In [ ]:
def create_event_for(role, day, additional='', name=''):
    """Create an icalendar event for this row for name and role"""
    event = Event()

    # Munge the role

    # Description should say who else is in department.
    description = role + \
        (': %s' % name if name != '' else '') + \
        (' (%s)' % additional if additional != '' else '')
    event.add('description', description)

    # Make the summary the same as the description
    event.add('summary', description)

    if 'start' in HOURS[role]:
        # If we have a start time in the HOURS dictionary for this role
        # - combine it with date
        event.add('dtstart',
                  datetime.combine(
                      day,
                      HOURS[role]['start']))
    else:
        # Otherwise just use the date
        event.add('dtstart', day)

    if 'duration' in HOURS[role]:
        event.add('duration', HOURS[role]['duration'])
    else:
        if (HOURS[role]['end'] > HOURS[role]['start']):
            event.add('dtend',
                      datetime.combine(
                          day,
                          HOURS[role]['end']))
        else:
            # OK so the end is before the start?
            # simply add a day on to the date and then combine
            event.add('dtend',
                      datetime.combine(
                          day + timedelta(days=1),
                          HOURS[role]['end']))

    event.add('dtstamp', datetime.now())
    event.add('location', 'At work')  # Set this to something useful
    event.add('uid', uuid.uuid4())
    return event


### `check_last_names`
We'll have to adjust `check_last_names` to be more like the simpler version in multi_rota1.py - but otherwise it's a simple copy.

In [19]:
def check_last_names(names_to_dates, directory, between):
    """Check from the previous run of this parser if there are new names,
    returns a dictionary of names to number of rows"""
    from os.path import exists, join
    from csv import DictReader, DictWriter

    last_names = {}
    # Read the last names
    if exists(join(directory, 'last_names.csv')):
        with open(join(directory, 'last_names.csv')) as f:
            r = DictReader(f)
            for row in r:
                last_names[row['name']] = int(row['number'])

    name_to_number_of_rows = {}
    with open(join(directory, 'last_names.csv'), 'w') as f:
        w = DictWriter(f, ['name', 'number'])
        w.writeheader()
        for name in names_to_dates:
            # number is the sum of rows for each role for this name, job pair
            number = len([day for day, _, _ in names_to_dates[name]
                         if day >= between[0] and day < between[1]])
            if name not in last_names:
                # We have a new name
                print('New name in rota: %s with %d rows' % (name, number))
            w.writerow({'name': name, 'number': number})
            name_to_number_of_rows[name] = number

    return name_to_number_of_rows

So if we put all those things in to a rota reader we get a working rota reader for the unusual rota [unusual1.py](unusual1.py)